In [1]:
# pip install gpxcsv

In [14]:
import pandas as pd
import numpy as np
import seaborn as sns
import folium
from gpxcsv import gpxtolist
from matplotlib import pyplot as plt
from matplotlib import colors
from os import listdir
from tqdm import tqdm
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from snippets import *

%matplotlib inline

In [15]:
DATA_PATH = "../../data"
RECORDS_PATH = f"{DATA_PATH}/records"
listdir(RECORDS_PATH)

['3_6.kml',
 '2021-12-14_18-47_Tue.gpx',
 '14_дек._2021_г._17_55_03.gpx',
 '3_6.gpx',
 '2021-12-14_17-54_Tue.gpx',
 '2021-12-14_23-25_Tue.gpx',
 '14_дек._2021_г._18_46_54.gpx']

In [25]:
df1 = pd.DataFrame(gpxtolist(f"{RECORDS_PATH}/3_6.gpx"))
display(df1.head(3))
display(df1.shape)

df2 = pd.DataFrame(gpxtolist(f"{RECORDS_PATH}/2021-12-14_17-54_Tue.gpx"))
display(df2.head(3))
display(df2.shape)

df3 = pd.DataFrame(gpxtolist(f"{RECORDS_PATH}/2021-12-14_18-47_Tue.gpx"))
display(df3.head(3))
display(df3.shape)

df4 = pd.DataFrame(gpxtolist(f"{RECORDS_PATH}/2021-12-14_23-25_Tue.gpx"))
display(df4.head(3))
display(df4.shape)


,lat,lon,name,ele
0,55.732227,37.639866,"От улица Зацепский Вал 3, Москва до Комсомольс...",123.0
1,55.686905,37.545494,"От улица Зацепский Вал 3, Москва до Комсомольс...",187.0
2,55.686990,37.545590,"От улица Зацепский Вал 3, Москва до Комсомольс...",187.0


(16140, 4)

,lat,lon,ele,time,hdop
0,55.968453,37.201455,129.5,2021-12-14T12:54:52Z,8.0
1,55.968540,37.201502,130.0,2021-12-14T12:54:54Z,8.0
2,55.968564,37.201421,130.0,2021-12-14T12:54:56Z,8.0


(681, 5)

,lat,lon,ele,time,hdop
0,55.857678,37.499142,130.0,2021-12-14T13:47:02Z,7.0
1,55.857785,37.498991,128.7,2021-12-14T13:47:04Z,7.0
2,55.857878,37.498891,128.7,2021-12-14T13:47:05Z,7.0


(1687, 5)

,lat,lon,ele,time,hdop
0,55.735635,37.488431,130.0,2021-12-14T18:25:29Z,10.0
1,55.735635,37.488431,130.0,2021-12-14T18:25:30Z,10.0
2,55.735635,37.488431,130.0,2021-12-14T18:25:31Z,14.0


(1280, 5)

In [26]:
tm_delta = pd.to_timedelta(range(len(df1)), unit="seconds")
df1["time"] = pd.to_datetime("now").round("s") + tm_delta
df1["gps_time"] = df1["time"]

df2["time"] = pd.to_datetime(df2["time"]).dt.tz_localize(None)
df2["gps_time"] = df2["time"]

df3["time"] = pd.to_datetime(df3["time"]).dt.tz_localize(None)
df3["time"] -= pd.to_timedelta(1, unit="days")
df3["gps_time"] = df3["time"]

df4["time"] = pd.to_datetime(df4["time"]).dt.tz_localize(None)
df4["time"] -= pd.to_timedelta(2, unit="days")
df4["gps_time"] = df4["time"]


display(df4.tail(1)["time"].values[0] < df3.head(1)["time"].values[0])
display(df3.tail(1)["time"].values[0] < df2.head(1)["time"].values[0])
display(df2.tail(1)["time"].values[0] < df1.head(1)["time"].values[0])


True

True

True

In [27]:
cols = ["time", "lat", "lon", "gps_time"]
df = pd.concat((df4[cols], df3[cols], df2[cols], df1[cols]), axis=0)
df

,time,lat,lon,gps_time
0,2021-12-12 18:25:29,55.735635,37.488431,2021-12-12 18:25:29
1,2021-12-12 18:25:30,55.735635,37.488431,2021-12-12 18:25:30
2,2021-12-12 18:25:31,55.735635,37.488431,2021-12-12 18:25:31
3,2021-12-12 18:25:32,55.735635,37.488431,2021-12-12 18:25:32
4,2021-12-12 18:25:33,55.735635,37.488430,2021-12-12 18:25:33
...,...,...,...,...
16135,2021-12-14 23:33:44,55.685080,37.902200,2021-12-14 23:33:44
16136,2021-12-14 23:33:45,55.685127,37.902275,2021-12-14 23:33:45
16137,2021-12-14 23:33:46,55.685165,37.902374,2021-12-14 23:33:46
16138,2021-12-14 23:33:47,55.685240,37.902485,2021-12-14 23:33:47


In [28]:
df.to_csv(f"{DATA_PATH}/track_20211214_211500.csv", index=False)

In [29]:
df_gps = df.copy()
df_gps["fraud"] = 1
df_gps["driver_hash"] = 10000000000001

In [30]:
# [None, -1, 0, 1]
FORCE_FRAUD = None

# Выбор случайного водителя
driver_gps, driver_hash, driver_fraud = sample_driver(df_gps, FORCE_FRAUD)
# driver_accel = df_accel[df_accel["driver_hash"] == driver_hash]

center = (driver_gps["lat"].mean(), driver_gps["lon"].mean())
m = folium.Map(location=center, zoom_start=10, legend=True)

# # Маршрут по данным акселерометра
# driver_accel = driver_accel.sort_values(by="time").reset_index(drop=True)
# map_driver_points(m, driver_accel, driver_hash, label="accel", color="blue")

# Маршрут по данным  GPS
driver_gps = driver_gps.sort_values(by="gps_time").reset_index(drop=True)
map_driver_points(m, driver_gps, driver_hash, label="gps")


folium.map.LayerControl('topleft', collapsed= False).add_to(m)
m